In [1]:
import cv2
from cvzone.PoseModule import PoseDetector
import math
import numpy as np
import time
import pickle
import mediapipe as mp
# Drawing helpers
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose
import pandas as pd

import os, sys
sys.path.append(os.path.abspath(".."))
from utils.common import load_model, save_model
import warnings
warnings.filterwarnings('ignore')

In [2]:
def rescale_frame(frame, percent=50):
    '''
    Rescale a frame to a certain percentage compare to its original frame
    '''
    width = int(frame.shape[1] * percent/ 100)
    height = int(frame.shape[0] * percent/ 100)
    dim = (width, height)
    return cv2.resize(frame, dim, interpolation =cv2.INTER_AREA)

In [3]:
IMPORTANT_LMS = [
    "NOSE",
    "LEFT_SHOULDER",
    "RIGHT_SHOULDER",
    "LEFT_HIP",
    "RIGHT_HIP",
    "LEFT_KNEE",
    "RIGHT_KNEE",
    "LEFT_ANKLE",
    "RIGHT_ANKLE"
]

# Tạo các cột cho dữ liệu đầu vào
HEADERS = ["label"]
for landmark in IMPORTANT_LMS:
    for dim in ['x', 'y', 'z']:
        HEADERS.append(f"{landmark.lower()}_{dim}")
        
# IMPORTANT_LMS = [element.lower() for element in IMPORTANT_LMS]

def extract_amd_recalculate_landmarks(pose_landmarks):
    """
    Tịnh tiến thân người vào giữa bức hình, đồng thời dời lại trục toạ độ
    """
    hip_center_x = float((pose_landmarks[23].x + pose_landmarks[24].x) / 2)
    hip_center_y = float((pose_landmarks[23].y + pose_landmarks[24].y) / 2)

    new_center = (0.5, 0.5)
    delta_x = new_center[0] - hip_center_x
    delta_y = new_center[1] - hip_center_y

    data = []
    for landmark in IMPORTANT_LMS:
        # Lấy ra id của key point trên cơ thể người
        key_point_id = mp_pose.PoseLandmark[landmark].value

        key_point = pose_landmarks[key_point_id]
        key_point.x += delta_x - 0.5
        key_point.y += delta_y -0.5
        data.append([key_point.x, key_point.y, key_point.z])

    return np.array(data).flatten().tolist()

In [4]:
def get_class(encode_label: float):
    return {
        0: "D",
        1: "M",
        2: "U",
    }.get(encode_label, "Unknown")

In [6]:
from utils.common import load_model

model = load_model("./best_models/SVC_model_side.pkl")
input_scaler = load_model("./best_models/input_scaler.pkl")

e:\Documents\Ki_6\PBL5\gymexcercisecorrection\.venv\Lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator StandardScaler from version 1.4.2 when using version 1.4.1.post1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


## Ver original

In [7]:
video_path = 'squat.mp4'
cap = cv2.VideoCapture(video_path)
detector = PoseDetector(detectionCon=0.7, trackCon=0.7)

# Creating Angle finder class
class angleFinder:
    def __init__(self, lmlist, p1, p2, p3, p4, p5, p6, p7, p8, drawPoints):
        self.lmlist = lmlist
        self.p1 = p1
        self.p2 = p2
        self.p3 = p3
        self.p4 = p4
        self.p5 = p5
        self.p6 = p6
        self.p7 = p7
        self.p8 = p8
        self.drawPoints = drawPoints

    # finding angles
    def angle(self):
        if len(self.lmlist) != 0:
            hipLeft = self.lmlist[self.p1][:2]
            kneeLeft = self.lmlist[self.p2][:2]
            ankleLeft = self.lmlist[self.p3][:2]
            hipRight = self.lmlist[self.p4][:2]
            kneeRight = self.lmlist[self.p5][:2]
            ankleRight = self.lmlist[self.p6][:2]
            leftShoulder = self.lmlist[self.p7][:2]
            rightShoulder = self.lmlist[self.p8][:2]

            if len(hipLeft) >= 2 and len(kneeLeft) >= 2 and len(ankleLeft) >= 2 and len(hipRight) >= 2 and len(kneeRight) >= 2 and len(
                    ankleRight) >= 2 and len(leftShoulder) >= 2 and len(rightShoulder) >= 2:
                x1, y1 = hipLeft[:2]
                x2, y2 = kneeLeft[:2]
                x3, y3 = ankleLeft[:2]
                x4, y4 = hipRight[:2]
                x5, y5 = kneeRight[:2]
                x6, y6 = ankleRight[:2]
                x7, y7 = leftShoulder[:2]
                x8, y8 = rightShoulder[:2]

                vertical_line_angle = 90  # Góc của đường thẳng đứng so với trục x

                # calculating angle for left and right hands
                leftHandAngle = math.degrees(math.atan2(y1 - y2, x1 - x2) - math.atan2(y1 - y2, 0))
                rightHandAngle = math.degrees(math.atan2(y4 - y5, x4 - x5) - math.atan2(y4 - y5, 0))

                leftBackAngle = math.degrees(math.atan2(y1 - y2, 0) - math.atan2(y7 - y1, x7 - x1))
                rightBackAngle = math.degrees(math.atan2(y4 - y5, 0) - math.atan2(y8 - y4, x8 - x4))

                p2_p3_angle = math.degrees(math.atan2(y3 - y2, x3 - x2))
                p5_p6_angle = math.degrees(math.atan2(y6 - y5, x6 - x5))
                # Tính góc giữa đường thẳng đứng và đường thẳng nối giữa đầu gối và mắt cá chân
                leftkneeAngleLineAngle = abs(vertical_line_angle - p2_p3_angle)
                rightkneeAngleLineAngle = abs(vertical_line_angle - p5_p6_angle)

                print (f"leftHandAngle: {leftHandAngle}")
                print (f"rightHandAngle: {rightHandAngle}")
                print (f"leftBackAngle: {leftBackAngle}")
                print (f"rightBackAngle: {rightBackAngle}")
                print (f"leftkneeAngleLineAngle: {leftkneeAngleLineAngle}")
                print (f"rightkneeAngleLineAngle: {rightkneeAngleLineAngle}")
                

                leftHandAngle = int(np.interp(leftHandAngle, [0, 95], [0, 100])) # Ánh xạ sang từ 0-95 về 0-100 vì góc càng nhỏ thì càng tiến về trạng thái s3
                rightHandAngle = int(np.interp(rightHandAngle, [0, 95], [0, 100])) # Ánh xạ sang từ 0-95 về 0-100 vì góc càng nhỏ thì càng tiến về trạng thái s3
                leftBackAngle = int(np.interp(leftBackAngle, [0, 95], [0, 100])) # Ánh xạ sang từ 0-95 về 0-100 vì góc càng nhỏ thì càng tiến về trạng thái s3
                rightBackAngle = int(np.interp(rightBackAngle, [0, 95], [0, 100])) # Ánh xạ sang từ 0-95 về 0-100 vì góc càng nhỏ thì càng tiến về trạng thái s3

                # drawing circles and lines on selected points
                if self.drawPoints:
                    cv2.circle(img, (x1, y1), 2, (0, 255, 0), 6)          
                    cv2.circle(img, (x2, y2), 2, (0, 255, 0), 6)           
                    cv2.circle(img, (x3, y3), 2, (0, 255, 0), 6)           
                    cv2.circle(img, (x4, y4), 2, (0, 255, 0), 6)           
                    cv2.circle(img, (x5, y5), 2, (0, 255, 0), 6)           
                    cv2.circle(img, (x6, y6), 2, (0, 255, 0), 6)            
                    cv2.circle(img, (x7, y7), 2, (0, 255, 0), 6)           
                    cv2.circle(img, (x8, y8), 2, (0, 255, 0), 6)

                    cv2.line(img, (x1, y1), (x2, y2), (0, 0, 255), 2)
                    cv2.line(img, (x2, y2), (x3, y3), (0, 0, 255), 2)
                    cv2.line(img, (x4, y4), (x5, y5), (0, 0, 255), 2)
                    cv2.line(img, (x5, y5), (x6, y6), (0, 0, 255), 2)
                    cv2.line(img, (x1, y1), (x4, y4), (0, 0, 255), 2)
                    cv2.line(img, (x1, y1), (x7, y7), (0, 0, 255), 2)
                    cv2.line(img, (x4, y4), (x8, y8), (0, 0, 255), 2)

                return [leftHandAngle, rightHandAngle, leftBackAngle, rightBackAngle, leftkneeAngleLineAngle, rightkneeAngleLineAngle]

# defining some variables
counter = 0
direction = 0
error1 = False
error2 = False
error3 = False
error4 = False
error5 = False

error1_start_time = None
error2_start_time = None
error3_start_time = None
error4_start_time = None
error5_start_time = None

with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while True:
        ret, img = cap.read()
        # img = cv2.resize(img, (640, 480))
        img = rescale_frame(img, percent=20)

        if not ret:
            print("Ignoring empty camera frame.")
            break
                
        # resize frame để tăng tốc độ xử lý
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        results = pose.process(img)

        if not results.pose_landmarks:
            print("No human found")
            continue

        # Cần khôi phục lại màu gốc của ảnh
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)

        key_points = extract_amd_recalculate_landmarks(results.pose_landmarks.landmark)
        X = pd.DataFrame([key_points], columns=HEADERS[1:])
        X = input_scaler.transform(X)

        predicted_class = model.predict(X)[0]
        predicted_class = get_class(model.predict(X)[0])

        # putting predicted class on the screen
        cv2.putText(img, f"State: {predicted_class}", (10, 420), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)


        detector.findPose(img, draw=0)
        lmList, bboxInfo = detector.findPosition(img, bboxWithHands=0, draw=False)
        
        angle1 = angleFinder(lmList, 23, 25, 27, 24, 26, 28, 11, 12, drawPoints=True)
        hands = angle1.angle()
        left, right, leftBackAngle, rightBackAngle, leftkneeAngleLineAngle, rightkneeAngleLineAngle = hands[0:]

        # Set lại giá trị ban đầu
        error1 = False
        error2 = False
        error3 = False
        error4 = False
        error5 = False

        # Counting number of squat
        if left > 75 and right > 75:
            if direction == 0:
                counter += 0.5
                direction = 1
        if left <= 70 and right <= 70:
            if direction == 1:
                counter += 0.5
                direction = 0

        current_time = time.time()

        if leftBackAngle >= 15 and rightBackAngle >= 15:
            error1 = True
            if error1_start_time is None:  # Nếu lỗi mới xuất hiện
                error1_start_time = current_time
        elif error1_start_time is not None:
            error1_start_time = None  # Reset thời gian nếu lỗi không còn

        if leftBackAngle >= 45 and rightBackAngle >= 45:
            error2 = True
            if error2_start_time is None:
                error2_start_time = current_time
        elif error2_start_time is not None:
            error2_start_time = None

        if leftBackAngle >= 30 and rightBackAngle >= 30 and left <= 80 and right <= 80:
            error3 = True
            if error3_start_time is None:
                error3_start_time = current_time
        elif error3_start_time is not None:
            error3_start_time = None

        if leftkneeAngleLineAngle >= 30 and rightkneeAngleLineAngle >= 30:
            error4 = True
            if error4_start_time is None:
                error4_start_time = current_time
        elif error4_start_time is not None:
            error4_start_time = None

        if left >= 95 and right >= 95:
            error5 = True
            if error5_start_time is None:
                error5_start_time = current_time
        elif error5_start_time is not None:
            error5_start_time = None

        # putting scores on the screen
        cv2.rectangle(img, (0, 0), (120, 120), (255, 0, 0), -1)
        cv2.putText(img, str(int(counter)), (1, 70), cv2.FONT_HERSHEY_SCRIPT_SIMPLEX, 1.6, (0, 0, 255), 6)

        # Draw errors
        if error1 and error1_start_time  and (current_time - error1_start_time) < 1000:
            cv2.rectangle(img, (380, 10), (630, 50), (0, 215, 215), -1)
            cv2.putText(img, "Bend Forward", (390, 30), cv2.FONT_HERSHEY_TRIPLEX , 0.7, (59, 59, 56), 3)
        if error2 and error2_start_time and (current_time - error2_start_time) < 1000:
            cv2.rectangle(img, (380, 60), (630, 100), (0, 215, 215), -1)
            cv2.putText(img, "Bend Backwards", (390, 80), cv2.FONT_HERSHEY_TRIPLEX , 0.7, (59, 59, 56), 3)
        if error3 and error3_start_time and (current_time - error3_start_time) < 1000:
            cv2.rectangle(img, (380, 110), (630, 150), (64, 64, 204), -1)
            cv2.putText(img, "Lower one's hips", (390, 130), cv2.FONT_HERSHEY_TRIPLEX, 0.7, (255, 255, 230), 3)
        if error4 and error4_start_time and (current_time - error4_start_time) < 1000:
            cv2.rectangle(img, (380, 160), (630, 200), (64, 64, 204), -1)
            cv2.putText(img, "Knee falling over toes", (390, 180), cv2.FONT_HERSHEY_TRIPLEX, 0.6, (255, 255, 230), 3)
        if error5 and error5_start_time and (current_time - error5_start_time) < 1000:
            cv2.rectangle(img, (380, 210), (630, 250), (204, 122, 0), -1)
            cv2.putText(img, "Deep squats", (390, 230), cv2.FONT_HERSHEY_TRIPLEX, 0.7, (255, 255, 230), 3)

        # Converting values for rectangles
        leftval = np.interp(left, [0, 100], [400, 200])
        rightval = np.interp(right, [0, 100], [400, 200])

        # For color changing
        value_left = np.interp(left, [0, 100], [0, 100])
        value_right = np.interp(right, [0, 100], [0, 100])

        # Drawing right rectangle and putting text
        cv2.putText(img, 'R', (24, 195), cv2.FONT_HERSHEY_DUPLEX, 1, (255, 0, 0), 5)
        cv2.rectangle(img, (8, 200), (50, 400), (0, 255, 0), 5)
        cv2.rectangle(img, (8, int(rightval)), (50, 400), (255, 0, 0), -1)

        # Drawing right rectangle and putting text
        cv2.putText(img, 'L', (604, 195), cv2.FONT_HERSHEY_DUPLEX, 1, (255, 0, 0), 5)
        cv2.rectangle(img, (582, 200), (632, 400), (0, 255, 0), 5)
        cv2.rectangle(img, (582, int(leftval)), (632, 400), (255, 0, 0), -1)

        # Tô màu đỏ khi góc đạt đến trạng thái s3
        if value_left > 75:
            cv2.rectangle(img, (582, int(leftval)), (632, 400), (0, 0, 255), -1)

        if value_right > 75:
            cv2.rectangle(img, (8, int(rightval)), (50, 400), (0, 0, 255), -1)

        cv2.imshow("Image", img)
        cv2.waitKey(1)

        # Nhấn q để thoát
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

e:\Documents\Ki_6\PBL5\gymexcercisecorrection\.venv\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(
e:\Documents\Ki_6\PBL5\gymexcercisecorrection\.venv\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(
e:\Documents\Ki_6\PBL5\gymexcercisecorrection\.venv\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(
e:\Documents\Ki_6\PBL5\gymexcercisecorrection\.venv\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(


leftHandAngle: 4.455602833326309
rightHandAngle: -1.63657704161672
leftBackAngle: 2.834111016306511
rightBackAngle: 3.044778444193991
leftkneeAngleLineAngle: 0.0
rightkneeAngleLineAngle: 3.0127875041833363
leftHandAngle: 4.455602833326309
rightHandAngle: -0.8184554616886155
leftBackAngle: 2.2679545359058437
rightBackAngle: 2.436648246810131
leftkneeAngleLineAngle: 0.0
rightkneeAngleLineAngle: 4.014175695411012


e:\Documents\Ki_6\PBL5\gymexcercisecorrection\.venv\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(
e:\Documents\Ki_6\PBL5\gymexcercisecorrection\.venv\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(
e:\Documents\Ki_6\PBL5\gymexcercisecorrection\.venv\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(
e:\Documents\Ki_6\PBL5\gymexcercisecorrection\.venv\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(
e:\Documents\Ki_6\PBL5\gymexcercisecorrection\.venv\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(
e:\Documents\Ki_6\PB

leftHandAngle: 3.715289105428768
rightHandAngle: -1.6135389328811875
leftBackAngle: 2.2679545359058437
rightBackAngle: 1.8279682443050145
leftkneeAngleLineAngle: 0.7073193685442618
rightkneeAngleLineAngle: 2.9609361341637452
leftHandAngle: 2.9356734464211645
rightHandAngle: -1.5911402711945943
leftBackAngle: 2.2679545359058437
rightBackAngle: 1.8279682443050145
leftkneeAngleLineAngle: 0.7161599454704088
rightkneeAngleLineAngle: 2.9609361341637452
leftHandAngle: 2.9356734464211645
rightHandAngle: -1.5911402711945943
leftBackAngle: 2.2679545359058437
rightBackAngle: 1.8087393224920651
leftkneeAngleLineAngle: 0.7161599454704088
rightkneeAngleLineAngle: 2.9609361341637452
leftHandAngle: 2.9356734464211645
rightHandAngle: -1.5911402711945943
leftBackAngle: 1.701354605326125
rightBackAngle: 1.2060487792199666
leftkneeAngleLineAngle: 0.7073193685442618
rightkneeAngleLineAngle: 4.014175695411012
leftHandAngle: 2.202598161765806
rightHandAngle: -1.6135389328811875
leftBackAngle: 1.1344216309770

e:\Documents\Ki_6\PBL5\gymexcercisecorrection\.venv\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(
e:\Documents\Ki_6\PBL5\gymexcercisecorrection\.venv\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(
e:\Documents\Ki_6\PBL5\gymexcercisecorrection\.venv\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(
e:\Documents\Ki_6\PBL5\gymexcercisecorrection\.venv\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(
e:\Documents\Ki_6\PBL5\gymexcercisecorrection\.venv\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(
e:\Documents\Ki_6\PB

...

e:\Documents\Ki_6\PBL5\gymexcercisecorrection\.venv\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(
e:\Documents\Ki_6\PBL5\gymexcercisecorrection\.venv\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(
e:\Documents\Ki_6\PBL5\gymexcercisecorrection\.venv\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(
e:\Documents\Ki_6\PBL5\gymexcercisecorrection\.venv\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(
e:\Documents\Ki_6\PBL5\gymexcercisecorrection\.venv\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(
e:\Documents\Ki_6\PB

leftHandAngle: -0.7538483330707685
rightHandAngle: -3.2245226065199164
leftBackAngle: -2.70062933739528
rightBackAngle: -7.815293546766876
leftkneeAngleLineAngle: 2.1475854282984983
rightkneeAngleLineAngle: 11.309932474020215
leftHandAngle: -0.7538483330707685
rightHandAngle: -3.2245226065199164
leftBackAngle: -2.70062933739528
rightBackAngle: -7.815293546766876
leftkneeAngleLineAngle: 2.121096396661457
rightkneeAngleLineAngle: 11.129189289611176
leftHandAngle: -0.7538483330707685
rightHandAngle: -3.2245226065199164
leftBackAngle: -2.70062933739528
rightBackAngle: -7.815293546766876
leftkneeAngleLineAngle: 2.121096396661457
rightkneeAngleLineAngle: 10.222168633636116


e:\Documents\Ki_6\PBL5\gymexcercisecorrection\.venv\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(
e:\Documents\Ki_6\PBL5\gymexcercisecorrection\.venv\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(
e:\Documents\Ki_6\PBL5\gymexcercisecorrection\.venv\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(
e:\Documents\Ki_6\PBL5\gymexcercisecorrection\.venv\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(


leftHandAngle: -0.7538483330707685
rightHandAngle: -3.2245226065199164
leftBackAngle: -2.6754266560291384
rightBackAngle: -7.815293546766876
leftkneeAngleLineAngle: 2.121096396661457
rightkneeAngleLineAngle: 10.222168633636116
leftHandAngle: 0.0
rightHandAngle: -2.419509216656339
leftBackAngle: -2.1409013663230896
rightBackAngle: -7.334378801416701
leftkneeAngleLineAngle: 2.827124578161275
rightkneeAngleLineAngle: 10.38885781546962
leftHandAngle: 0.7440592028887014
rightHandAngle: -2.419509216656339
leftBackAngle: -0.5354589855645314
rightBackAngle: -7.891702428235702
leftkneeAngleLineAngle: 2.827124578161275
rightkneeAngleLineAngle: 9.462322208025611


e:\Documents\Ki_6\PBL5\gymexcercisecorrection\.venv\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(
e:\Documents\Ki_6\PBL5\gymexcercisecorrection\.venv\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(
e:\Documents\Ki_6\PBL5\gymexcercisecorrection\.venv\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(
e:\Documents\Ki_6\PBL5\gymexcercisecorrection\.venv\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(
e:\Documents\Ki_6\PBL5\gymexcercisecorrection\.venv\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(
e:\Documents\Ki_6\PB